In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

torch.manual_seed(1)

In [3]:
context_size = 2
embedding_dim = 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

print(len(test_sentence))

115


In [4]:
# context_size=2이므로 앞 두 단어가 주어졌을 때 세번째 단어를 예측하는 작업
trigrams = [([test_sentence[i], test_sentence[i+1]], test_sentence[i+2])
           for i in range(len(test_sentence)-2)]
print(trigrams)

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege'), (['shall', 'besiege'], 'thy'), (['besiege', 'thy'], 'brow,'), (['thy', 'brow,'], 'And'), (['brow,', 'And'], 'dig'), (['And', 'dig'], 'deep'), (['dig', 'deep'], 'trenches'), (['deep', 'trenches'], 'in'), (['trenches', 'in'], 'thy'), (['in', 'thy'], "beauty's"), (['thy', "beauty's"], 'field,'), (["beauty's", 'field,'], 'Thy'), (['field,', 'Thy'], "youth's"), (['Thy', "youth's"], 'proud'), (["youth's", 'proud'], 'livery'), (['proud', 'livery'], 'so'), (['livery', 'so'], 'gazed'), (['so', 'gazed'], 'on'), (['gazed', 'on'], 'now,'), (['on', 'now,'], 'Will'), (['now,', 'Will'], 'be'), (['Will', 'be'], 'a'), (['be', 'a'], "totter'd"), (['a', "totter'd"], 'weed'), (["totter'd", 'weed'], 'of'), (['weed', 'of'], 'small'), (['of', 'small'], 'worth'), (['small', 'worth'], 'held:'), (['worth', 'held:'], 'Then'), (['held:', 'Then'], 'being'), (['Then', 'being'], 'asked,'), (['being', 'asked,'], 'wher

In [5]:
vocab = set(test_sentence)
word2idx = {word: idx for idx, word in enumerate(vocab)}
print(word2idx)

{'This': 0, 'an': 1, 'the': 2, 'thriftless': 3, 'thy': 4, 'Where': 5, 'warm': 6, 'see': 7, 'worth': 8, 'trenches': 9, 'deep': 10, "feel'st": 11, 'lies,': 12, 'much': 13, 'Thy': 14, 'all-eating': 15, 'cold.': 16, 'Were': 17, "totter'd": 18, 'And': 19, 'fair': 20, 'use,': 21, 'sunken': 22, 'days;': 23, 'old,': 24, 'besiege': 25, 'lusty': 26, 'couldst': 27, 'blood': 28, 'shame,': 29, 'praise': 30, 'own': 31, 'being': 32, 'If': 33, 'thine': 34, 'made': 35, 'on': 36, 'where': 37, 'more': 38, 'in': 39, 'proud': 40, 'small': 41, 'to': 42, "beauty's": 43, 'new': 44, 'by': 45, "'This": 46, 'a': 47, 'asked,': 48, 'livery': 49, 'beauty': 50, 'gazed': 51, 'praise.': 52, 'answer': 53, 'weed': 54, "youth's": 55, 'count,': 56, 'thou': 57, 'his': 58, 'dig': 59, 'when': 60, "excuse,'": 61, 'held:': 62, 'say,': 63, 'so': 64, 'mine': 65, 'How': 66, 'my': 67, 'were': 68, 'of': 69, 'To': 70, 'Proving': 71, 'child': 72, 'treasure': 73, 'succession': 74, 'field,': 75, 'eyes,': 76, 'Will': 77, 'old': 78, 'all

In [6]:
class NGramModel(nn.Module):
    # vocab_size: 총 단어의 갯수
    # embedding_dim: embedding output의 차원
    # context_size: 총 몇 개의 input을 가지고 output을 도출?
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size*embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, x):
        embeded = self.embedding(x).view(1, -1)
        out = F.relu(self.linear1(embeded))
        out = self.linear2(out)
        prob = F.log_softmax(out, dim=1)
        return prob

In [7]:
criterion = nn.NLLLoss()
model = NGramModel(len(vocab), embedding_dim, context_size)
print(model)

NGramModel(
  (embedding): Embedding(97, 10)
  (linear1): Linear(in_features=20, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=97, bias=True)
)


In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [10]:
for epoch in range(10):
    losses = []
    for context, target  in trigrams:
        # input(context)을 word2idx을 이용해 모델로 넘겨줄 작업
        context_idxs = [word2idx[w] for w in context]
        context_idxs = torch.tensor(context_idxs, dtype=torch.long)
        target = torch.tensor([word2idx[target]], dtype=torch.long)
        
#         print(context_idxs.shape)
#         print(target.shape)
        
        optimizer.zero_grad()
        outputs = model(context_idxs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
    print("[%d/%d] loss:%3f" % (epoch+1, 10, np.mean(losses)))

[1/10] loss:2.550497
[2/10] loss:2.316757
[3/10] loss:2.085636
[4/10] loss:1.861997
[5/10] loss:1.650320
[6/10] loss:1.454676
[7/10] loss:1.277305
[8/10] loss:1.119649
[9/10] loss:0.981355
[10/10] loss:0.860985
